In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
import os
from tensorflow.keras.callbacks import ModelCheckpoint

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


## Attempt 1
drop noise features, classification at 1000

In [2]:
# drop noisy features
attempt_one = application_df.drop(["INCOME_AMT","EIN","NAME","ASK_AMT"], axis = 1)

# Look at APPLICATION_TYPE value counts for binning
ap_counts_one = attempt_one.APPLICATION_TYPE.value_counts()

# Determine which values to replace if counts are less than ...?
replace_application_one = list(ap_counts_one[ap_counts_one < 500].index)

# Replace in dataframe
for app in replace_application_one:
    attempt_one.APPLICATION_TYPE = attempt_one.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
attempt_one.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [3]:
# Look at CLASSIFICATION value counts for binning
classification_counts = attempt_one['CLASSIFICATION'].value_counts()

# Determine which values to replace if counts are less than ..?
replace_class = list(classification_counts[classification_counts < 1000].index)

# Replace in dataframe
for cls in replace_class:
    attempt_one.CLASSIFICATION = attempt_one.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
attempt_one.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [4]:
# Generate our categorical variable lists
application_cat_attempt_one = attempt_one.dtypes[attempt_one.dtypes == "object"].index.tolist()
application_cat_attempt_one

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'SPECIAL_CONSIDERATIONS']

In [5]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df_one = pd.DataFrame(enc.fit_transform(attempt_one[application_cat_attempt_one]))

# Add the encoded variable names to the dataframe
encode_df_one.columns = enc.get_feature_names_out(application_cat_attempt_one)
encode_df_one.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


In [6]:
# Merge one-hot encoded features and drop the originals
attempt_one = attempt_one.merge(encode_df_one, left_index=True,right_index=True)
attempt_one = attempt_one.drop(application_cat_attempt_one, axis = 1)
attempt_one.head()

,STATUS,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
1,1,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,1,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
3,1,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4,1,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


In [7]:
attempt_one.columns

Index(['STATUS', 'IS_SUCCESSFUL', 'APPLICATION_TYPE_Other',
       'APPLICATION_TYPE_T10', 'APPLICATION_TYPE_T19', 'APPLICATION_TYPE_T3',
       'APPLICATION_TYPE_T4', 'APPLICATION_TYPE_T5', 'APPLICATION_TYPE_T6',
       'APPLICATION_TYPE_T7', 'APPLICATION_TYPE_T8',
       'AFFILIATION_CompanySponsored', 'AFFILIATION_Family/Parent',
       'AFFILIATION_Independent', 'AFFILIATION_National', 'AFFILIATION_Other',
       'AFFILIATION_Regional', 'CLASSIFICATION_C1000', 'CLASSIFICATION_C1200',
       'CLASSIFICATION_C2000', 'CLASSIFICATION_C2100', 'CLASSIFICATION_C3000',
       'CLASSIFICATION_Other', 'USE_CASE_CommunityServ', 'USE_CASE_Heathcare',
       'USE_CASE_Other', 'USE_CASE_Preservation', 'USE_CASE_ProductDev',
       'ORGANIZATION_Association', 'ORGANIZATION_Co-operative',
       'ORGANIZATION_Corporation', 'ORGANIZATION_Trust',
       'SPECIAL_CONSIDERATIONS_N', 'SPECIAL_CONSIDERATIONS_Y'],
      dtype='object')

In [8]:
# Split our preprocessed data into our features and target arrays
y = attempt_one['IS_SUCCESSFUL'].values
X = attempt_one.drop(['IS_SUCCESSFUL'],axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [9]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [10]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                2720      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5,181
Trainable params: 5,181
Non-trainable params: 0
_________________________________________________________________


In [11]:
# Define the checkpoint path and filenames
os.makedirs("checkpoints_opt/",exist_ok=True)
checkpoint_path = "checkpoints_opt/weights.{epoch:02d}.hdf5"

In [12]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [13]:
# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5)

In [14]:
# Train the model
fit_model = nn.fit(X_train, y_train,epochs=50)

Epoch 1/50
804/804 [==============================] - 1s 639us/step - loss: 0.5790 - accuracy: 0.7163
Epoch 2/50
804/804 [==============================] - 0s 605us/step - loss: 0.5634 - accuracy: 0.7253
Epoch 3/50
804/804 [==============================] - 0s 591us/step - loss: 0.5612 - accuracy: 0.7239
Epoch 4/50
804/804 [==============================] - 0s 604us/step - loss: 0.5602 - accuracy: 0.7257
Epoch 5/50
804/804 [==============================] - 0s 597us/step - loss: 0.5596 - accuracy: 0.7260
Epoch 6/50
804/804 [==============================] - 0s 599us/step - loss: 0.5588 - accuracy: 0.7257
Epoch 7/50
804/804 [==============================] - 0s 593us/step - loss: 0.5585 - accuracy: 0.7271
Epoch 8/50
804/804 [==============================] - 0s 609us/step - loss: 0.5580 - accuracy: 0.7280
Epoch 9/50
804/804 [==============================] - 0s 597us/step - loss: 0.5581 - accuracy: 0.7275
Epoch 10/50
804/804 [==============================] - 0s 598us/step - loss: 0.557

In [15]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 2.1545 - accuracy: 0.7074 - 182ms/epoch - 679us/step
Loss: 2.154522180557251, Accuracy: 0.7074052691459656


## Attempt 2
Remove income_amt, ask_amt as well as Affiliation, increase application type cutoff to 700, used tanh for second activation

In [16]:
# drop noisy features
attempt_two = application_df.drop(["EIN","NAME","INCOME_AMT","AFFILIATION","ASK_AMT"], axis = 1)

# Look at APPLICATION_TYPE value counts for binning
ap_counts_two = attempt_two.APPLICATION_TYPE.value_counts()

# Determine which values to replace if counts are less than ...?
replace_application_two = list(ap_counts_two[ap_counts_two < 700].index)

# Replace in dataframe
for app in replace_application_two:
    attempt_two.APPLICATION_TYPE = attempt_two.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
attempt_two.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
Other      804
T8         737
T7         725
Name: APPLICATION_TYPE, dtype: int64

In [17]:
# Look at CLASSIFICATION value counts for binning
classification_counts = attempt_two['CLASSIFICATION'].value_counts()

# Determine which values to replace if counts are less than ..?
replace_class_two = list(classification_counts[classification_counts < 750].index)

# Replace in dataframe
for cls in replace_class_two:
    attempt_two.CLASSIFICATION = attempt_two.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
attempt_two.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [18]:
# Generate our categorical variable lists
application_cat_attempt_two = attempt_two.dtypes[attempt_two.dtypes == "object"].index.tolist()
application_cat_attempt_two

['APPLICATION_TYPE',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'SPECIAL_CONSIDERATIONS']

In [19]:
# Fit and transform the OneHotEncoder using the categorical variable list
encode_df_two = pd.DataFrame(enc.fit_transform(attempt_two[application_cat_attempt_two]))

# Add the encoded variable names to the dataframe
encode_df_two.columns = enc.get_feature_names_out(application_cat_attempt_two)
encode_df_two.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,CLASSIFICATION_C1000,CLASSIFICATION_C1200,...,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


In [20]:
# Merge one-hot encoded features and drop the originals
attempt_two = attempt_two.merge(encode_df_two, left_index=True,right_index=True)
attempt_two = attempt_two.drop(application_cat_attempt_two, axis = 1)
attempt_two.head()

,STATUS,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,...,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
1,1,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,1,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
3,1,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4,1,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


In [21]:
# Split our preprocessed data into our features and target arrays
y = attempt_two['IS_SUCCESSFUL'].values
X = attempt_two.drop(['IS_SUCCESSFUL'],axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [22]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [23]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30

nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Output layer
nn2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 80)                2240      
                                                                 
 dense_4 (Dense)             (None, 30)                2430      
                                                                 
 dense_5 (Dense)             (None, 1)                 31        
                                                                 
Total params: 4,701
Trainable params: 4,701
Non-trainable params: 0
_________________________________________________________________


In [24]:
# Compile the model
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [25]:
# Train the model
fit_model = nn2.fit(X_train, y_train,epochs=50)

Epoch 1/50
804/804 [==============================] - 1s 575us/step - loss: 0.6285 - accuracy: 0.6373
Epoch 2/50
804/804 [==============================] - 0s 569us/step - loss: 0.6177 - accuracy: 0.6439
Epoch 3/50
804/804 [==============================] - 0s 573us/step - loss: 0.6163 - accuracy: 0.6483
Epoch 4/50
804/804 [==============================] - 0s 603us/step - loss: 0.6149 - accuracy: 0.6475
Epoch 5/50
804/804 [==============================] - 0s 612us/step - loss: 0.6145 - accuracy: 0.6474
Epoch 6/50
804/804 [==============================] - 0s 607us/step - loss: 0.6136 - accuracy: 0.6496
Epoch 7/50
804/804 [==============================] - 0s 613us/step - loss: 0.6134 - accuracy: 0.6506
Epoch 8/50
804/804 [==============================] - 0s 579us/step - loss: 0.6129 - accuracy: 0.6516
Epoch 9/50
804/804 [==============================] - 1s 628us/step - loss: 0.6123 - accuracy: 0.6512
Epoch 10/50
804/804 [==============================] - 0s 601us/step - loss: 0.612

In [26]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 1.0879 - accuracy: 0.5452 - 164ms/epoch - 612us/step
Loss: 1.0879040956497192, Accuracy: 0.5451894998550415


## Attempt 3
Add layer, reset to cutoff at 500
remove income_amt and affiliation


In [27]:
# drop noisy features
attempt_three = application_df.drop(["EIN","NAME"], axis = 1)

# Look at APPLICATION_TYPE value counts for binning
ap_counts_three = attempt_three.APPLICATION_TYPE.value_counts()

# Determine which values to replace if counts are less than ...?
replace_application_three = list(ap_counts_three[ap_counts_three < 500].index)

# Replace in dataframe
for app in replace_application_three:
    attempt_three.APPLICATION_TYPE = attempt_three.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
attempt_three.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [28]:
# Look at CLASSIFICATION value counts for binning
classification_counts = attempt_three['CLASSIFICATION'].value_counts()

# Determine which values to replace if counts are less than ..?
replace_class_three = list(classification_counts[classification_counts < 500].index)

# Replace in dataframe
for cls in replace_class_three:
    attempt_three.CLASSIFICATION = attempt_three.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
attempt_three.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [29]:
# Generate our categorical variable lists
application_cat_attempt_three = attempt_three.dtypes[attempt_three.dtypes == "object"].index.tolist()

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df_three = pd.DataFrame(enc.fit_transform(attempt_three[application_cat_attempt_three]))

# Add the encoded variable names to the dataframe
encode_df_three.columns = enc.get_feature_names_out(application_cat_attempt_three)

# Merge one-hot encoded features and drop the originals
attempt_three = attempt_three.merge(encode_df_three, left_index=True,right_index=True)
attempt_three = attempt_three.drop(application_cat_attempt_three, axis = 1)
attempt_three.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [30]:
# Split our preprocessed data into our features and target arrays
y = attempt_three['IS_SUCCESSFUL'].values
X = attempt_three.drop(['IS_SUCCESSFUL'],axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30

nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Output layer
nn3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn3.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 80)                3600      
                                                                 
 dense_7 (Dense)             (None, 30)                2430      
                                                                 
 dense_8 (Dense)             (None, 1)                 31        
                                                                 
Total params: 6,061
Trainable params: 6,061
Non-trainable params: 0
_________________________________________________________________


In [31]:
# Compile the model
nn3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [32]:
# Train the model
fit_model = nn3.fit(X_train, y_train,epochs=50)

Epoch 1/50
804/804 [==============================] - 1s 600us/step - loss: 0.6954 - accuracy: 0.5335
Epoch 2/50
804/804 [==============================] - 1s 696us/step - loss: 0.6916 - accuracy: 0.5288
Epoch 3/50
804/804 [==============================] - 1s 674us/step - loss: 0.6916 - accuracy: 0.5327
Epoch 4/50
804/804 [==============================] - 1s 639us/step - loss: 0.6915 - accuracy: 0.5329
Epoch 5/50
804/804 [==============================] - 0s 583us/step - loss: 0.6917 - accuracy: 0.5335
Epoch 6/50
804/804 [==============================] - 0s 587us/step - loss: 0.6916 - accuracy: 0.5335
Epoch 7/50
804/804 [==============================] - 0s 578us/step - loss: 0.6916 - accuracy: 0.5304
Epoch 8/50
804/804 [==============================] - 1s 673us/step - loss: 0.6916 - accuracy: 0.5331
Epoch 9/50
804/804 [==============================] - 0s 590us/step - loss: 0.6917 - accuracy: 0.5300
Epoch 10/50
804/804 [==============================] - 0s 579us/step - loss: 0.691

In [33]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.7867 - accuracy: 0.3820 - 176ms/epoch - 656us/step
Loss: 0.7866901159286499, Accuracy: 0.3820408284664154


In [34]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")